In [ ]:
%matplotlib widget

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

from lvmdrp import log, path, __version__ as drpver
from lvmdrp.utils import metadata as md

from lvmdrp.core.constants import ARC_LAMPS, SPEC_CHANNELS
from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod


# style.use(['dark_background', 'ggplot', 'fast'])

In [ ]:
# define list of lamps
lamps = [lamp.lower() for lamp in ARC_LAMPS]
lamps.remove("xenon")
print(f"{lamps = }")
# initialize lamps status (OFF)
lamps_status = dict.fromkeys(lamps, False)

In [ ]:
master_paths = [os.path.join(root, file) for root, _, files in os.walk(os.getenv("LVM_SPECTRO_REDUX")) for file in files if file.startswith("lvm-xmarc_") and file.endswith(".fits")]
master_arcs = md.extract_metadata(kind="master", frames_paths=master_paths)
master_arcs = master_arcs.loc[master_arcs.mjd == 60145]
master_arcs.sort_values("camera", inplace=True)

In [ ]:
ref_fibers = {"r2": 330, "r1": 319, "r3": 319, "z1": 319, "z2": 319, "z3": 319, "b3": 319, "b2": 319, "b1": 319}


mwaves = []
for camera in sorted(ref_fibers):
    
    if camera[0] == "b":
        lamp = "hgne"
        marc = master_arcs.query("camera == @camera & hgne").squeeze()
    else:
        lamp = "neon"
        marc = master_arcs.query("camera == @camera & neon").squeeze()
    
    xpath = path.full("lvm_master", drpver=drpver, kind=f"xmarc_{lamp}", **marc.to_dict())
    mwave = path.full("lvm_master", drpver=drpver, kind=f"mwave_{lamp}", **marc.to_dict())
    mlsf = path.full("lvm_master", drpver=drpver, kind=f"mlsf_{lamp}", **marc.to_dict())
    wpath = path.full("lvm_master", drpver=drpver, kind=f"wmarc_{lamp}", **marc.to_dict())

    # determine wavelength solutions
    rssMethod.determine_wavelength_solution(in_arcs=xpath, out_wave=mwave, out_lsf=mlsf, ref_fiber=ref_fibers.get(marc.camera, 319), aperture=10, cc_correction=True,
                                            poly_disp=3, poly_fwhm=2, poly_cros=2, flux_min=1e-12, fwhm_max=5, rel_flux_limits=[0.001, 1e12], display_plots=True)
    
    mwaves.append(mwave)
    # create pixel table
    iwave, fwave = SPEC_CHANNELS[marc.camera[0]]
    wpath = path.full("lvm_master", drpver=drpver, kind=f"wmarc_{lamp}", **marc.to_dict())
    rssMethod.create_pixel_table(in_rss=xpath, out_rss=wpath, arc_wave=mwave, arc_fwhm=mlsf)
    # resample
    rssMethod.resample_wavelength(in_rss=wpath, out_rss=wpath, method="linear", disp_pix=0.5, start_wave=iwave, end_wave=fwave, err_sim=10, parallel=0)
    # combine channel
    # combine spectrograph

In [ ]:
from astropy.io import fits

# save coefficients table
coeffs = pd.DataFrame(columns=sorted(ref_fibers))
for mwave in mwaves:
    wave = fits.open(mwave)
    camera = os.path.basename(mwave).replace(".fits", "").split("-")[-1]
    coeff = wave[1].data

    coeffs[camera] = coeff[coeff.shape[0]//2]

coeffs.to_csv(f"data_waves/coeffs_fib{coeff.shape[0]//2}_neon.csv")

In [ ]:
fiberflats = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="xfiberflat", exptime="x").sort_values("camera")

for _, fiberflat in fiberflats.iterrows():
    print(fiberflat)
    
    rssMethod.createPixTable_drp(
        in_rss=fiberflat.path,
        out_rss=out_calib_path.format(mjd=fiberflat.mjd, kind="wfiberflat", camera=fiberflat.camera, exptime=fiberflat.exptime),
        arc_wave=out_arc_path.format(mjd=fiberflat.mjd, kind="wave", camera=fiberflat.camera, lamps=LAMPS),
        arc_fwhm=out_arc_path.format(mjd=fiberflat.mjd, kind="lsf", camera=fiberflat.camera, lamps=LAMPS)
    )

    wave_range = CHANNEL_WL[fiberflat.camera[0]]
    rssMethod.resampleWave_drp(
        in_rss=out_calib_path.format(mjd=fiberflat.mjd, kind="wfiberflat", camera=fiberflat.camera, exptime=fiberflat.exptime),
        out_rss=out_calib_path.format(mjd=fiberflat.mjd, kind="hfiberflat", camera=fiberflat.camera, exptime=fiberflat.exptime),
        start_wave=wave_range[0], end_wave=wave_range[1], disp_pix=1.0, method="linear",
        err_sim=10, parallel="auto", extrapolate=True
    )

In [ ]:
arcs = get_masters_metadata(path_pattern=out_arc_path, mjd="super", kind="xarc", lamps=LAMPS).sort_values("camera")

for _, arc in arcs.iterrows():
    print(arc)
    
    rssMethod.createPixTable_drp(
        in_rss=arc.path,
        out_rss=out_arc_path.format(mjd=arc.mjd, kind="warc", camera=arc.camera, lamps=LAMPS),
        arc_wave=out_arc_path.format(mjd=arc.mjd, kind="wave", camera=arc.camera, lamps=LAMPS),
        arc_fwhm=out_arc_path.format(mjd=arc.mjd, kind="lsf", camera=arc.camera, lamps=LAMPS)
    )

    wave_range = CHANNEL_WL[arc.camera[0]]
    rssMethod.resampleWave_drp(
        in_rss=out_arc_path.format(mjd=arc.mjd, kind="warc", camera=arc.camera, lamps=LAMPS),
        out_rss=out_arc_path.format(mjd=arc.mjd, kind="harc", camera=arc.camera, lamps=LAMPS),
        start_wave=wave_range[0], end_wave=wave_range[1], disp_pix=1.0, method="linear",
        err_sim=10, parallel="auto", extrapolate=True
    )